# 상수 텐서와 변수

In [6]:
import tensorflow as tf
import numpy as np

In [3]:
# 모두 1 또는 0인 텐서
x = tf.ones(shape = (2, 1))
print(x)

x = tf.zeros(shape = (2, 1))
print(x)

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


In [5]:
# 랜덤 텐서
x = tf.random.normal(shape = (3, 1), mean = 0, stddev = 1)
print(x)

x = tf.random.uniform(shape = (3, 1), minval = 0, maxval= 1)
print(x)

tf.Tensor(
[[-0.09817437]
 [ 0.271745  ]
 [-0.42499426]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[0.9570458 ]
 [0.9684441 ]
 [0.13380218]], shape=(3, 1), dtype=float32)


In [8]:
# 넘파이 배열에 값 할당하기
x = np.ones(shape = (2, 1))
x[0, 0] = 0
x

array([[0.],
       [1.]])

In [9]:
# 텐서플로 텐서에는 값을 할당하지 못한다.
x = tf.ones(shape = (2, 1))
x[0, 0] = 0.

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [10]:
# 텐서플로 변수 만들기
v = tf.Variable(initial_value = tf.random.normal(shape = (3, 1)))
print(v)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[ 0.15931413],
       [ 0.12317395],
       [-0.69024724]], dtype=float32)>


In [12]:
# 텐서플로 변수에 값 할당하기
v.assign(tf.ones((3, 1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.]], dtype=float32)>

In [13]:
# 변수 일부에 값 할당하기
v[0, 0].assign(3.)

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [1.],
       [1.]], dtype=float32)>

In [15]:
v.assign_add(tf.ones((3, 1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[5.],
       [3.],
       [3.]], dtype=float32)>

# 텐서 연산 : 텐서플로에서 수학 계산하기